# 效能指標-預測最低價格

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 讀取原始數據
file_path = '/Users/yuching/Documents/專題/merge_and_cleaned/final_data/long_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與商務艙的數據
economy_class_data = data[data["艙等（主航段）"] == "經濟艙"].copy()
business_class_data = data[data["艙等（主航段）"] == "商務艙"].copy()

# One-hot encoding（指定分類欄位）
categorical_cols = ['出發時段', '出發機場代號', '抵達時段', '航空聯盟組合', '航空公司組合',
                   '航空聯盟', '停靠站數量', '機型分類', '假期', '飛行時間兩段分類', '是否為平日']
economy_class_data = pd.get_dummies(economy_class_data, columns=categorical_cols, drop_first=True)
business_class_data = pd.get_dummies(business_class_data, columns=categorical_cols, drop_first=True)

# 取兩個數據集共同擁有的欄位
common_cols = list(set(economy_class_data.columns) & set(business_class_data.columns))
economy_class_data = economy_class_data[common_cols]
business_class_data = business_class_data[common_cols]

# 標準化數值變數
num_cols = ['停留時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_class_data[num_cols] = scaler.fit_transform(economy_class_data[num_cols])
business_class_data[num_cols] = scaler.transform(business_class_data[num_cols])

# 建模特徵欄位（根據關鍵字）
target_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空聯盟組合_', '航空公司組合_',
                   '航空聯盟_', '停靠站數量_', '機型分類_', '假期_', '飛行時間兩段分類_', '是否為平日_']
economy_dummy_cols = [col for col in economy_class_data.columns if any(keyword in col for keyword in target_keywords)]
business_dummy_cols = [col for col in business_class_data.columns if any(keyword in col for keyword in target_keywords)]
economy_feature_cols = economy_dummy_cols + num_cols
business_feature_cols = business_dummy_cols + num_cols

# --------------------------
# 最佳參數 (long_最低價格_log)
# --------------------------
best_params_econ = {
    'RandomForest': {'n_estimators': 200, 'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 1},
    'SVR'         : {'C': 6.0, 'epsilon': 0.1, 'gamma': 'scale'},
    'XGBoost'     : {'n_estimators': 250, 'learning_rate': 0.15, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8}
}
best_params_biz = {
    'RandomForest': {'n_estimators': 250, 'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 1},
    'SVR'         : {'C': 6.0, 'epsilon': 0.1, 'gamma': 'scale'},
    'XGBoost'     : {'n_estimators': 250, 'learning_rate': 0.15, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8}
}

seed = 123

def train_and_predict_model(ModelClass, params, X_train, X_test, y_train, y_test, label):
    # 如果是 XGB，加上 n_jobs、tree_method、verbosity
    if ModelClass is XGBRegressor:
        model = ModelClass(**params, random_state=seed, n_jobs=-1, tree_method='hist', verbosity=0)
    else:
        model = ModelClass(**params, random_state=seed, n_jobs=-1) \
                if ModelClass is RandomForestRegressor \
                else ModelClass(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    print(f"📌 {label} 測試集  MSE: {mse:.4f}  R²: {r2:.4f}")
    return y_pred

# === 經濟艙模型訓練 ===
print("====== 經濟艙模型訓練 ======")
X_e = economy_class_data[economy_feature_cols]
y_e = economy_class_data["最低價格_log"]
X_tr_e, X_te_e, y_tr_e, y_te_e = train_test_split(X_e, y_e, test_size=0.3, random_state=seed)

rf_pred_e   = train_and_predict_model(RandomForestRegressor, best_params_econ['RandomForest'], X_tr_e, X_te_e, y_tr_e, y_te_e, "RandomForest")
svr_pred_e  = train_and_predict_model(SVR, best_params_econ['SVR'], X_tr_e, X_te_e, y_tr_e, y_te_e, "SVR")
xgb_pred_e  = train_and_predict_model(XGBRegressor, best_params_econ['XGBoost'], X_tr_e, X_te_e, y_tr_e, y_te_e, "XGBoost")

economy_results = economy_class_data.loc[X_te_e.index].copy()
economy_results["預測最低價格_rf"]  = np.exp(rf_pred_e)
economy_results["預測最低價格_svr"] = np.exp(svr_pred_e)
economy_results["預測最低價格_xgb"] = np.exp(xgb_pred_e)

# === 商務艙模型訓練 ===
print("\n====== 商務艙模型訓練 ======")
X_b = business_class_data[business_feature_cols]
y_b = business_class_data["最低價格_log"]
X_tr_b, X_te_b, y_tr_b, y_te_b = train_test_split(X_b, y_b, test_size=0.3, random_state=seed)

rf_pred_b   = train_and_predict_model(RandomForestRegressor, best_params_biz['RandomForest'], X_tr_b, X_te_b, y_tr_b, y_te_b, "RandomForest")
svr_pred_b  = train_and_predict_model(SVR, best_params_biz['SVR'], X_tr_b, X_te_b, y_tr_b, y_te_b, "SVR")
xgb_pred_b  = train_and_predict_model(XGBRegressor, best_params_biz['XGBoost'], X_tr_b, X_te_b, y_tr_b, y_te_b, "XGBoost")

business_results = business_class_data.loc[X_te_b.index].copy()
business_results["預測最低價格_rf"]  = np.exp(rf_pred_b)
business_results["預測最低價格_svr"] = np.exp(svr_pred_b)
business_results["預測最低價格_xgb"] = np.exp(xgb_pred_b)

# === 計算並顯示平均節省比例 ===
def calc_eff(df, col):
    return ((df["隨機購買平均價格"] - df[col]) / df["隨機購買平均價格"]).mean() * 100

print("\n====== 平均節省比例 ======")
print(f"經濟艙 RF  : {calc_eff(economy_results, '預測最低價格_rf'):.2f}%")
print(f"經濟艙 SVR : {calc_eff(economy_results, '預測最低價格_svr'):.2f}%")
print(f"經濟艙 XGB : {calc_eff(economy_results, '預測最低價格_xgb'):.2f}%")
print(f"商務艙 RF  : {calc_eff(business_results, '預測最低價格_rf'):.2f}%")
print(f"商務艙 SVR : {calc_eff(business_results, '預測最低價格_svr'):.2f}%")
print(f"商務艙 XGB : {calc_eff(business_results, '預測最低價格_xgb'):.2f}%")

====== 經濟艙模型訓練 ======
📌 RandomForest 測試集  MSE: 0.0250  R²: 0.9186
📌 SVR 測試集  MSE: 0.0465  R²: 0.8488
📌 XGBoost 測試集  MSE: 0.0259  R²: 0.9158

====== 商務艙模型訓練 ======
📌 RandomForest 測試集  MSE: 0.0114  R²: 0.9139
📌 SVR 測試集  MSE: 0.0227  R²: 0.8279
📌 XGBoost 測試集  MSE: 0.0127  R²: 0.9038

====== 平均節省比例 ======
經濟艙 RF  : 17.39%
經濟艙 SVR : 16.67%
經濟艙 XGB : 17.54%
商務艙 RF  : 2.27%
商務艙 SVR : 1.27%
商務艙 XGB : 2.36%
